In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../data/tirol_obituaries_deduped.csv', delimiter = ',')

In [3]:
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df.head()

,date,year,week,municipaly,district,hash
0,2020-12-26,2020,52,Inzing,Innsbruck-Land,-1627801591
1,2020-12-26,2020,52,Telfs,Innsbruck-Land,1227327702
2,2020-12-26,2020,52,Fieberbrunn,Kitzbühel,1742164729
3,2020-12-26,2020,52,Schwoich,Kufstein,782141748
4,2020-12-25,2020,52,Kirchdorf in Tirol,Kitzbühel,637367255


In [4]:
df_count_by_day = (pd.to_datetime(df['date'])
       .dt.floor('d')
       .value_counts()
       .rename_axis('date')
       .reset_index(name='count'))

df_count_by_day = df_count_by_day.sort_values('date')

df_count_by_day.tail(10)

,date,count
145,2020-12-17,20
1459,2020-12-18,9
184,2020-12-19,20
170,2020-12-20,20
1439,2020-12-21,9
563,2020-12-22,16
391,2020-12-23,17
1098,2020-12-24,12
1668,2020-12-25,2
1659,2020-12-26,4


In [5]:
df_by_day_by_location = df.groupby(['date','municipaly']).size().reset_index(name='count')
df_by_day_by_location.tail(10)

,date,municipaly,count
19928,2020-12-24,Obertilliach,1
19929,2020-12-24,Reutte,1
19930,2020-12-24,Vils,1
19931,2020-12-24,Zirl,1
19932,2020-12-25,Kirchdorf in Tirol,1
19933,2020-12-25,Nassereith,1
19934,2020-12-26,Fieberbrunn,1
19935,2020-12-26,Inzing,1
19936,2020-12-26,Schwoich,1
19937,2020-12-26,Telfs,1


In [6]:
df_by_day_by_district = df.groupby(['date','district']).size().reset_index(name='count')
df_by_day_by_district.tail(10)

,date,district,count
10829,2020-12-24,Kufstein,2
10830,2020-12-24,Landeck,1
10831,2020-12-24,Lienz,3
10832,2020-12-24,Reutte,3
10833,2020-12-24,Schwaz,1
10834,2020-12-25,Imst,1
10835,2020-12-25,Kitzbühel,1
10836,2020-12-26,Innsbruck-Land,2
10837,2020-12-26,Kitzbühel,1
10838,2020-12-26,Kufstein,1


In [7]:
zillertal = [
    "Aschau im Zillertal",
    "Brandberg",
    "Bruck am Ziller",
    "Finkenberg",
    "Fügen",
    "Fügenberg",
    "Gerlos",
    "Gerlosberg",
    "Hainzenberg",
    "Hart im Zillertal",
    "Hippach",
    "Kaltenbach",
    "Mayrhofen",
    "Ramsau im Zillertal",
    "Ried im Zillertal",
    "Rohrberg",
    "Schlitters",
    "Schwendau",
    "Strass im Zillertal",
    "Stumm",
    "Stummerberg",
    "Tux",
    "Uderns",
    "Zell am Ziller",
    "Zellberg"
]

df_zillertal = df[df['municipaly'].isin(zillertal)]
df_zillertal.head()

,date,year,week,municipaly,district,hash
7,2020-12-24,2020,52,Hainzenberg,Schwaz,479576250
30,2020-12-23,2020,52,Zell am Ziller,Schwaz,1065796954
44,2020-12-22,2020,52,Zell am Ziller,Schwaz,-387599008
65,2020-12-20,2020,51,Schwendau,Schwaz,-1886661432
71,2020-12-20,2020,51,Schwendau,Schwaz,-1433569288


In [8]:
import time

idx = pd.date_range('06-01-2016', '20-12-2020')

df_count_by_day = df_count_by_day.set_index('date')
df_count_by_day = df_count_by_day.reindex(idx, fill_value=0)
df_count_by_day.index.name = 'date'
df_count_by_day = df_count_by_day.reset_index()

df_count_by_day.tail(30)

,date,count
1634,2020-11-21,18
1635,2020-11-22,21
1636,2020-11-23,20
1637,2020-11-24,23
1638,2020-11-25,19
1639,2020-11-26,26
1640,2020-11-27,18
1641,2020-11-28,24
1642,2020-11-29,19
1643,2020-11-30,25


In [9]:
import altair as alt
alt.data_transformers.disable_max_rows()
day = alt.Chart(df_count_by_day, width=900, height=200).properties(
    title='Todesfälle pro Tag mit 7-day moving average'
).mark_circle(
    color='lightblue',
    size=16
).encode(
    x='date:T',
    y='count:Q',
    tooltip=['date:T', 'count:Q']
)

# .transform_timeunit(
#     yearmonth='yearmonth(date)'
# ).transform_aggregate(
#     sum_deaths='sum(count)',
#     sum_buckets='distinct(yearmonth)'
# ).transform_calculate(
#     avg='datum.sum_deaths / datum.sum_buckets'
# )


#   "transform": [
#     {"timeUnit": "yearmonthdate", "field": "date", "as": "date"},
#     {
#       "aggregate": [{"op": "mean", "field": "temp", "as": "temp"}],
#       "groupby": ["date"]
#     },
#     {"calculate": "day(datum.date) == 0", "as": "sundays"},
#     {
#       "window": [{"op": "sum", "field": "sundays", "as": "week"}],
#       "sort": "date"
#     }
#   ],

ma = alt.Chart(df_count_by_day, width=900, height=200
              ).mark_line(
    color='#336699',
    size=1.5
).transform_window(
    rolling_mean='mean(count)',
    frame=[-7, 0]
).encode(
    x='date:T',
    y='rolling_mean:Q',
    tooltip=['date:T', 'rolling_mean:Q']
)

(day + ma).interactive()

alt.LayerChart(...)

In [10]:
df_by_location = df.groupby(['municipaly']).size().reset_index(name='count')
df_by_location = df_by_location.sort_values('count', ascending=False)
df_by_location.head(10)

,municipaly,count
88,Innsbruck,3013
114,Kufstein,877
126,Lienz,692
261,Wattens,672
197,Schwaz,510
271,Wörgl,435
72,Hall in Tirol,387
110,Kitzbühel,370
215,St. Johann in Tirol,358
118,Landeck,339


In [11]:
df_by_district = df.groupby(['district']).size().reset_index(name='count')
df_by_district = df_by_district.sort_values('count', ascending=False)
df_by_district.head(10)

,district,count
1,Innsbruck-Land,5223
4,Kufstein,3628
2,Innsbruck-Stadt,3013
8,Schwaz,2821
3,Kitzbühel,2384
6,Lienz,2222
5,Landeck,1609
0,Imst,1581
7,Reutte,1037


In [12]:
alt.renderers.enable('html')
alt.data_transformers.disable_max_rows()
chartDistrictsBars = alt.Chart().mark_bar().encode(
    x=alt.X('yearmonth(date):T', title=None),
    y=alt.Y('sum(count):Q', title=None),
    tooltip=['yearmonth(date):T', 'sum(count):Q']
).properties(
    width=225,
    height=100,
)

chartDistrictsLine = alt.Chart().mark_rule(color='lightblue').encode(
    y='avg:Q',
    size=alt.SizeValue(1.5),
    tooltip=['avg:Q']
).transform_timeunit(
    yearmonth='yearmonth(date)'
).transform_aggregate(
    sum_deaths='sum(count)',
    sum_buckets='distinct(yearmonth)'
).transform_calculate(
    avg='datum.sum_deaths / datum.sum_buckets'
)

chartDistricts = alt.layer(chartDistrictsBars, data=df_by_day_by_district).facet(
    facet=alt.Facet('district:N', title=None),
    columns=3,
    title={
      "text": ["Todesanzeigen Tirol pro Bundesland pro Monat seit Juni 2016"], 
      "subtitle": [
          "Quelle: https://github.com/walterra/covid-19-tirol-ds"
      ],
      "color": "black",
      "subtitleColor": "gray"
    },
)


chartDistricts

alt.FacetChart(...)

In [13]:
bars = alt.Chart(df_by_location.head(20)).mark_bar().encode(
    x='sum(count):Q',
    y=alt.Y('municipaly:N', sort='-x')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='count:Q'
)

bars + text


alt.LayerChart(...)

In [14]:
# with open("../docs/data/vega_districts.json", "w") as jsonFile:
#     json.dump(json.loads(chartDistricts.to_json()), jsonFile, indent=4, sort_keys=True)

In [15]:
alt.renderers.enable('html')
alt.data_transformers.disable_max_rows()

def municipalyChart(district='Landeck'):
    df_sm = df[df['district']==district]

    df_sm = df_sm.groupby(['date','district','municipaly']).size().reset_index(name='count')

    bars = alt.Chart().mark_bar().encode(
        x=alt.X('yearmonth(date):T', title=None),
        y=alt.Y('sum(count):Q', title=None),
        tooltip=['yearmonth(date):T', 'sum(count):Q']
    ).properties(
        width=135,
        height=80,
    )
    
    line = alt.Chart().mark_rule(color='lightblue').encode(
        y='avg:Q',
        size=alt.SizeValue(1.5),
        tooltip=['avg:Q']
    ).transform_timeunit(
        yearmonth='yearmonth(date)'
    ).transform_aggregate(
        sum_deaths='sum(count)',
        sum_buckets='distinct(yearmonth)'
    ).transform_calculate(
        avg='datum.sum_deaths / datum.sum_buckets'
    )
    
    return alt.layer(bars, data=df_sm).facet(
        facet=alt.Facet('municipaly:N', title=None),
        columns=6,
        padding=0,
        title={
          "text": ["Zeitverlauf Todesanzeigen im Bezirk "+district+" pro Gemeinde pro Monat"], 
          "subtitle": [
#               "Durchschnitt pro Monat in Hellblau",
              "Quelle: https://github.com/walterra/covid-19-tirol-ds"
          ],
          "color": "black",
          "subtitleColor": "gray"
        },
    )

municipalyChart('Landeck')

alt.FacetChart(...)

In [16]:
# districts = {
#     "imst": "Imst",
#     "innsbruck-land": "Innsbruck-Land",
#     "innsbruck-stadt": "Innsbruck-Stadt",
#     "kitzbuehel": "Kitzbühel",
#     "kufstein": "Kufstein",
#     "landeck": "Landeck",
#     "reutte": "Reutte",
#     "schwaz": "Schwaz",
#     "lienz": "Lienz"
# }

# for d in districts:
#     chart = municipalyChart(districts[d])
#     with open("../docs/data/vega_" + d + ".json", "w") as jsonFile:
#         json.dump(json.loads(chart.to_json()), jsonFile, indent=4, sort_keys=True)

In [17]:
alt.renderers.enable('html')
alt.data_transformers.disable_max_rows()

def municipalyChartZillertal():
    df_sm = df_zillertal

    df_sm = df_sm.groupby(['date','district','municipaly']).size().reset_index(name='count')

    bars = alt.Chart().mark_bar().encode(
        x=alt.X('yearmonth(date):T', title=None),
        y=alt.Y('sum(count):Q', title=None),
        tooltip=['yearmonth(date):T', 'sum(count):Q']
    ).properties(
        width=135,
        height=80,
    )
    
    line = alt.Chart().mark_rule(color='lightblue').encode(
        y='avg:Q',
        size=alt.SizeValue(1.5),
        tooltip=['avg:Q']
    ).transform_timeunit(
        yearmonth='yearmonth(date)'
    ).transform_aggregate(
        sum_deaths='sum(count)',
        sum_buckets='distinct(yearmonth)'
    ).transform_calculate(
        avg='datum.sum_deaths / datum.sum_buckets'
    )
    
    return alt.layer(bars, data=df_sm).facet(
        facet=alt.Facet('municipaly:N', title=None),
        columns=6,
        padding=0,
        title={
          "text": ["Zeitverlauf Todesanzeigen im pro Zillertal-Gemeinde pro Monat"], 
          "subtitle": [
#               "Durchschnitt pro Monat in Hellblau",
              "Quelle: https://github.com/walterra/covid-19-tirol-ds"
          ],
          "color": "black",
          "subtitleColor": "gray"
        },
    )

municipalyChartZillertal()

alt.FacetChart(...)